In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. quick check on the dataset

In [ ]:
path = "../input/tabular-playground-series-nov-2021"
sample_submission_df = pd.read_csv("{}//sample_submission.csv".format(path))
test_df = pd.read_csv("{}//test.csv".format(path))
train_df = pd.read_csv("{}//train.csv".format(path))

In [ ]:
## Check the shape of submission data sheet
sample_submission_df

## -> note: need to remove index column for upload

In [ ]:
#drop "id" columns
try:
    train_X = train_df.drop("id", axis=1)
    test_X = test_df.drop("id", axis=1)
except KeyError as e:
    print(e)

train_X_wid = train_df
test_X_wid = test_df

In [ ]:
# test_X
train_X

In [ ]:
## check target population
ratio_label = round(float(train_X["target"].value_counts()[0]/train_X["target"].value_counts()[1]),3)
print(f"ratio:{ratio_label}")
train_df["target"].hist()

##-> almost 50/50 on train_df

In [ ]:
#drop "target" columns for building models
try :
    train_X = train_X.drop("target", axis=1)
except KeyError as e:
    print(e)

train_y = train_df["target"]

In [ ]:
folds=3
kf = KFold(n_splits=folds)
lgb_params = {"objective":"binary",
              "metrics":"auc",
              "random_seed":1234}

## 2. Building a LightGBM Model for base line
Of cousese I belive I can create better model with dataset which doesn't have "id" columns
since "id" have nothing to do with target lavel. 
But I want to make sure how much "id" affects to the score.
Fist of all, I will create a model with dataset which doesn't have "id" columns

### 2-1. using dataset without "id" column 


In [ ]:
##
models=[]
for train_index, valid_index in kf.split(train_X):
    X_train = train_X.iloc[train_index]
    y_train = train_y.iloc[train_index]
    X_valid = train_X.iloc[valid_index]
    y_valid = train_y.iloc[valid_index]
    
    lgb_train= lgb.Dataset(X_train, y_train)
    lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    
    model_lgb = lgb.train(lgb_params, 
                          lgb_train, 
                          valid_sets=lgb_valid, 
                          num_boost_round=100,
                          early_stopping_rounds=20,
                          verbose_eval=10
                         )
                          
    y_pred = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)
#     print(accuracy_score(np.round(y_pred), y_valid))
    
    models.append(model_lgb)

In [ ]:
preds=[]
for model in models:
    lgb.plot_importance(model, importance_type="gain", max_num_features=15)
    pred = model_lgb.predict(test_X)
    preds.append(pred)

In [ ]:
preds_array = np.array(preds)
preds_mean = np.mean(preds_array, axis=0)
# preds_mean
preds_int = (preds_mean > 0.5).astype(int)